# 震相拾取与记分流程

In [1]:
import sys
sys.path.append("/home/disk/wd_black/wzm/Sustech_Pulse/model/DiTing/ditingbench_preview")
sys.path.append("/home/disk/wd_black/wzm/Sustech_Pulse/src/finetune")

from diting_lsm_model import load_DiTing100M_preveiw #, DiTing_EQDet_PhasePick_predict , DiTing_EQDet_PhasePick_predict_fastV2_multievents , DiTing_EQDet_PhasePick_predict_fastV2
from dpk_utils import DiTing_EQDet_PhasePick_predict_fastV2, DiTing_EQDet_PhasePick_predict_fastV2_multievents
from diting_all import  DiTing_EQDet_phase_TTA_predict_2 ,DiTing_EQDet_phase_TTA_predict ,DiTing_EQDet_phase_TTA_predict_3

from pathlib import Path
import obspy
import numpy as np
import numpy.ma as ma
import torch
sys.path.append("/home/disk/disk02/wzm/Sustech_Pulse/tutorial/diting_tutorial_001")


import matplotlib.pyplot as plt

dpk_model_path = '/home/disk/wd_black/wzm/Sustech_Pulse/model/DiTing/DiTing0.1B-preview/DiTing0.1B-preview.pth'
dpk_device = 'cuda:0'
dpk_model = load_DiTing100M_preveiw( weight_path=dpk_model_path, device=dpk_device)

/home/zbk/anaconda3/envs/ditinglsm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/zbk/anaconda3/envs/ditinglsm/lib/python3.10/site-packages/timm/models/layers/__init__.py:48: FutureWarning: Importing from timm.models.layers is deprecated, please import via timm.layers
  warnings.warn(f"Importing from {__name__} is deprecated, please import via timm.layers", FutureWarning)


Please 'pip install apex'
=> loading checkpoint '/home/disk/wd_black/wzm/Sustech_Pulse/model/DiTing/DiTing0.1B-preview/DiTing0.1B-preview.pth'
[linear_probe/none]Trainable parameters: 13613663
[linear_probe/none]Trainable parameters: 23279007
Sequential(
  (0): ViTAdapter(
    (backbone): PatchTSTEncoder(
      (W_p): Conv1d(3, 512, kernel_size=(50,), stride=(50,))
      (dropout): Dropout(p=0.0, inplace=False)
      (encoder): TSTEncoder(
        (layers): ModuleList(
          (0-23): 24 x TSTEncoderLayer(
            (self_attn): MultiheadAttention_ROPE(
              (q_proj): Linear(in_features=512, out_features=512, bias=True)
              (k_proj): Linear(in_features=512, out_features=512, bias=True)
              (v_proj): Linear(in_features=512, out_features=512, bias=True)
              (o_proj): Linear(in_features=512, out_features=512, bias=True)
              (rotary_emb): LlamaRotaryEmbedding()
              (sdp_attn): ScaledDotProductAttention(
                (attn_dr

# 记分函数（答案可重复，扣分按整体）

In [ ]:
from typing import List

def calculate_score(
    P_answer_list: List[float],
    S_answer_list: List[float],
    P_res_list: List[float],
    S_res_list: List[float],
) -> float:
    """
    计分规则：
    - P波：误差<=0.1 记1分；在(0.1, 1]间线性从1降至0；>1 记0分
    - S波：误差<=0.2 记1分；在(0.2, 2]间线性从1降至0；>2 记0分
    - 匹配策略：遍历“答案”，每次从“结果”中选与该答案最接近的一条进行打分（选过的结果不再复用）
    - 惩罚：若结果（预测）数量多于答案数量，超出每一个扣 0.5 分；总分不低于 0
    - 最终分数为 P 分 + S 分（分别独立计分与扣分）
    """
    def _err_to_score(err: float, tight: float, loose: float) -> float:
        if err <= tight:
            return 1.0
        if err <= loose:
            return 1.0 - (err - tight) / (loose - tight)
        return 0.0

    def _score_one_phase(answer_list: List[float],
                         result_list: List[float],
                         tight: float,
                         loose: float) -> float:
        # 不修改原列表
        remaining = sorted(result_list)
        total = 0.0
        for a in sorted(answer_list):
            if not remaining:
                # 没有可匹配的结果，记0分，继续
                continue
            # 找到离当前答案最近的结果并匹配
            idx = min(range(len(remaining)), key=lambda i: abs(remaining[i] - a))
            err = abs(remaining[idx] - a)
            total += _err_to_score(err, tight, loose)
            # 该结果已使用，移除，避免复用
            #remaining.pop(idx)

        # 超检惩罚（仅针对多出来的未被匹配的结果）
        extra = len(answer_list) - len(result_list)
        if extra < 0:
            extra = 0
        penalty = 0

        return max(0.0, total - penalty)  # 扣到0则不得分

    p_score = _score_one_phase(P_answer_list, P_res_list, tight=0.1, loose=1.0)
    s_score = _score_one_phase(S_answer_list, S_res_list, tight=0.2, loose=2.0)

    return p_score + s_score

# 计算主程序

读取月份

In [2]:
#输入月份，自动更改目录
month = '09'

#数据读取
answer_dict = dict()
answer_file_path = '/home/disk/wd_black/wzm/Sustech_Pulse/test_data/%s-an/T1.an' %month
for line in open(answer_file_path).readlines():
    line = line.strip()
    key = line.split(':')[0].strip()
    P_line = line.split('P :')[1].split(' : S : ')[0]
    S_line = line.split(' : S : ')[1]
    answer_dict[key] = [P_line, S_line]

开始计算

In [3]:
# 目前还是没有做TTA的情况，因为上次的TTA示例代码是针对单个事件的，后面需要做针对多个事件的
# Test-time augmentation

score = 0
pscore = 0
sscore = 0
P_dev_list = []
S_dev_list = []
ans_number=0
res_number=0
base_exam_path = '/home/disk/wd_black/wzm/Sustech_Pulse/test_data/%s-exam/T1/{}' %month
out_dir = "/home/disk/wd_black/wzm/Sustech_Pulse/src/finetune/zbk/dpk_output_%s.txt" %month

with open(out_dir,"w") as f:
    f.write("i    P_answer   S_answer   P_predict   S_predict   \n")

keys = list(answer_dict.keys())#[:1] 
for key in keys:
    P_line = answer_dict[key][0]
    if ';' in P_line:
        P_answer_list = []
        for t_split in P_line.split(';'):
            P_answer_list.append(float(t_split))
    else:
        P_answer_list = [float(P_line)]
    S_line = answer_dict[key][1]
    if ';' in S_line:
        S_answer_list = []
        for t_split in S_line.split(';'):
            S_answer_list.append(float(t_split))
    else:
        S_answer_list = [float(S_line)]

    test_data = base_exam_path.format(key)
    # print(test_data)
    st = obspy.read(test_data, format="MSEED", stream=True)
    # st.detrend('demean') #去均值

    # st.detrend('linear') #去趋势
    # st.filter('bandpass',freqmin=0.005, freqmax=50.0) 
    
    if st[0].stats.sampling_rate != 100.0:
        st.resample(100.0)

    results = {}
    station_list = []
    for tr in st:
        station_list.append(tr.stats.station)
    station_list = list(set(station_list))
    for sta in station_list:
        results[sta] = {}
        results[sta]['P'] = []
        results[sta]['S'] = []
        results[sta]['P_for_compare'] = []
        results[sta]['S_for_compare'] = []
        
        input_st = st.select(station=sta)
        #对长的波形做multi event repeat (或者事件比较多，就走多事件的分支)
        if input_st[0].stats.endtime - input_st[0].stats.starttime > 8*60:
            input_st = input_st.filter('bandpass',freqmin=3, freqmax=50.0) 
            events = DiTing_EQDet_PhasePick_predict_fastV2_multievents(input_st, dpk_device, dpk_model, window_length=10000, step_size=3000, p_th=0.1, s_th=0.1, det_th=0.20, batch_size=100, max_repeat=7)
        else:
            input_st = input_st.detrend('demean')
            events = DiTing_EQDet_PhasePick_predict_fastV2(input_st, dpk_device, dpk_model, window_length=10000, step_size=3000, p_th=0.1, s_th=0.1, det_th=0.30, batch_size=100, return_confidence=False)
        for t_event in events:
            try:
                results[sta]['P'].append(str(input_st[0].stats.starttime + t_event[1][0][0]/100.0))
                results[sta]['P_for_compare'].append( t_event[1][0][0]/100.0 )
            except:
                print(key,"no P")

                pass
            try:
                results[sta]['S'].append(str(input_st[0].stats.starttime + t_event[2][0][0]/100.0))
                results[sta]['S_for_compare'].append( t_event[2][0][0]/100.0 )
            except:
                print(key,"no S")
                pass
        with open(out_dir, "a") as f:
            f.write(f"{P_answer_list} {results[sta]['P_for_compare']} {S_answer_list} {results[sta]['S_for_compare']}\n")
        p_score,s_score =calculate_score(P_answer_list, S_answer_list, results[sta]['P_for_compare'], results[sta]['S_for_compare'])

        pscore += p_score
        sscore += s_score
        get_score=p_score+s_score
        res_number +=len(results[sta]['P_for_compare'])+len(results[sta]['S_for_compare'])
        ans_number +=len(P_answer_list)+len(S_answer_list)
        #if get_score<len(P_answer_list)+len(S_answer_list)*0.9:
        print(key,get_score, "detect sum" , len(results[sta]['P_for_compare'])+len(results[sta]['S_for_compare']) ,"answer sum",  len(P_answer_list)+len(S_answer_list) )

            # st = input_st
            # plt.figure(figsize=(24, 6))
            # for tdx, tr in enumerate(st):
            #     plt.subplot(len(st), 1, tdx + 1)
            #     plt.plot(tr.times(), tr.data, color='k')
            #     plt.title(tr.stats.station)

            #     for filter_P in P_answer_list:
            #         plt.axvline(x=input_st[0].stats.starttime + (np.mean(filter_P)), color='red', linestyle='--', label='P')
            #     for filter_S in S_answer_list:
            #         plt.axvline(x=input_st[0].stats.starttime + (np.mean(filter_S)), color='green', linestyle='--', label='S')
            #     for GT_P in results[sta]['P_for_compare']:
            #         plt.axvline(x=input_st[0].stats.starttime + GT_P, color='blue', linestyle=':', label='GT P')
            #     for GT_S in results[sta]['S_for_compare']:
            #         plt.axvline(x=input_st[0].stats.starttime + GT_S, color='orange', linestyle=':', label='GT S')
            # plt.legend()
            # plt.xlabel('Time (s)')
            # plt.ylabel('Amplitude')
            # plt.tight_layout()
            # plt.suptitle(key+" get:"+str(get_score)+"/"+str( len(results[sta]['P_for_compare'])+len(results[sta]['P_for_compare']))+" should: "+str( len(P_answer_list)+len(S_answer_list)))
            # plt.savefig(f"/home/disk/wd_black/wzm/Sustech_Pulse/src/finetune/wzm/e10/{key}.png")
            # plt.close()
            # plt.figure(figsize=(24, 6))
            # for tdx, tr in enumerate(st):
            #     plt.subplot(len(st), 1, tdx + 1)
            #     plt.plot(tr.times(), tr.data, color='k')
            #     plt.xlim(30, 33)
            #     plt.title(tr.stats.station)
            #     plt.axvline(x=input_st[0].stats.starttime + (np.mean(filter_P)), color='red', linestyle='--', label='P')
            #     plt.axvline(x=input_st[0].stats.starttime + (np.mean(filter_S)), color='green', linestyle='--', label='S')
            #     plt.axvline(x=input_st[0].stats.starttime + GT_P, color='blue', linestyle=':', label='GT P')
            #     plt.axvline(x=input_st[0].stats.starttime + GT_S, color='orange', linestyle=':', label='GT S')

            # plt.legend()
            # plt.xlabel('Time (s)')
            # plt.ylabel('Amplitude')
            # plt.tight_layout()
            # plt.show()


    # print('cur score: {:.2f}'.format(score))
score=pscore+sscore
if res_number > ans_number* 1.05:  # 超过5%
    score=pscore+sscore-math.ceil((res_number-ans_number*1.05))*0.5

print(score)

/home/zbk/anaconda3/envs/ditinglsm/lib/python3.10/site-packages/obspy/signal/filter.py:62: UserWarning: Selected high corner frequency (50.0) of bandpass is at or above Nyquist (50.0). Applying a high-pass instead.
  warnings.warn(msg)


On 0


/home/disk/wd_black/wzm/Sustech_Pulse/src/finetune/dpk_utils.py:605: RuntimeWarning: invalid value encountered in divide
  confidence = confidence / count
/home/zbk/anaconda3/envs/ditinglsm/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/zbk/anaconda3/envs/ditinglsm/lib/python3.10/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


On 1
On 2
On 3
On 4
On 5
On 6
T1.E.Q0001.mseed no S
T1.E.Q0001.mseed no S
T1.E.Q0001.mseed no S
T1.E.Q0001.mseed no S
T1.E.Q0001.mseed no S
T1.E.Q0001.mseed no S
T1.E.Q0001.mseed no S
T1.E.Q0001.mseed no S
T1.E.Q0001.mseed no S
T1.E.Q0001.mseed no S
T1.E.Q0001.mseed no S
T1.E.Q0001.mseed no S
T1.E.Q0001.mseed no S
T1.E.Q0001.mseed no S
T1.E.Q0001.mseed no S
T1.E.Q0001.mseed no S
T1.E.Q0001.mseed no S
T1.E.Q0001.mseed no S
T1.E.Q0001.mseed no S
T1.E.Q0001.mseed no S
T1.E.Q0001.mseed no S
T1.E.Q0001.mseed no S
T1.E.Q0001.mseed no S
T1.E.Q0001.mseed no S
T1.E.Q0001.mseed no S
T1.E.Q0001.mseed no S
T1.E.Q0001.mseed no S
T1.E.Q0001.mseed no S
T1.E.Q0001.mseed no S


NameError: name 'calculate_score' is not defined

In [ ]:
print(res_number)
print(ans_number)
print(pscore+sscore)
print(score)